<a href="https://colab.research.google.com/github/Rob-A-B/wildfire-hotspots/blob/main/src/queimadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dask dask[dataframe] pyarrow

In [ ]:
!pip install cudf-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [ ]:
import os, datetime
from google.colab import files
import shutil, os, datetime
import os, glob, datetime
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import re
from google.colab import drive
import gc

import dask.dataframe as dd
import dask_cudf

In [ ]:
hoje = datetime.date.today()
BASE   = "/content/projeto_pcb"
BRONZE_RAW = f"{BASE}/dados/bronze/raw"                   # onde guardamos os arquivos "brutos" por partição
BRONZE_CUR = f"{BASE}/dados/bronze/current"               # dataset consolidado (único arquivo)
os.makedirs(BRONZE_RAW, exist_ok=True)
os.makedirs(BRONZE_CUR, exist_ok=True)


DRIVE_CSV_PATH = "/content/drive/My Drive/8_periodo/Big Data/datasets"
os.makedirs(DRIVE_CSV_PATH, exist_ok=True)

# BRONZE

In [ ]:
# 1) Descobrir todos os CSVs diretamente de um link ou ID de pasta compartilhada do Google Drive


shared_folder_link_or_id = 'https://drive.google.com/drive/folders/1hRgLe3t-zGgWqFH7ZAtW0m6XQdgOW3bh?usp=sharing'

download_dir = f"{DRIVE_CSV_PATH}/downloaded_csvs"
os.makedirs(download_dir, exist_ok=True)

print(f"📦 Attempting to download all files from shared Google Drive folder: {shared_folder_link_or_id}")

try:
    # Use gdown to download all files from the folder
    # The --folder flag indicates that the provided link/ID is for a folder
    # The -O flag specifies the output directory
    !gdown --no-cookies --no-check-certificate --folder "{shared_folder_link_or_id}" -O "{download_dir}"

    # After downloading, list the files in the download directory to process them
    arquivos_csv = [f for f in os.listdir(download_dir) if f.endswith(".csv") and os.path.isfile(os.path.join(download_dir, f))]
    print(f"📦 Downloaded {len(arquivos_csv)} CSVs to {download_dir}.\n")

except Exception as e:
    print(f"⚠️ Error during download: {e}")
    print("Please ensure gdown is installed (`!pip install gdown`), the folder link/ID is correct, and the folder is shared with 'Anyone with the link'.")
    arquivos_csv = [] # Ensure arquivos_csv is empty if download fails


# 2) Mover/copy to Bronze/raw partitioned and read into pandas DataFrames

for nome in arquivos_csv:
    caminho_src = os.path.join(download_dir, nome)

    # extrai AAAAMM do nome (ex.: focos_mensal_br_202311.csv)
    m = re.search(r'(\d{6})', nome)
    aaaamm = m.group(1) if m else f"{hoje.year}{hoje:%m}"
    ano, mes = aaaamm[:4], aaaamm[4:]

    # destino particionado (save as parquet in bronze raw)
    dest_dir = os.path.join(BRONZE_RAW, f"ano={ano}", f"mes={mes}")
    os.makedirs(dest_dir, exist_ok=True)
    nome_parquet = nome.replace(".csv", ".parquet")
    caminho_dst_parquet = os.path.join(dest_dir, nome_parquet)


    try:
        df_temp = pd.read_csv(caminho_src)
        df_temp["origem_arquivo"] = nome

        # salva como Parquet em bronze raw
        df_temp.to_parquet(caminho_dst_parquet, index=False)
        print(f"✅ Processado e salvo como Parquet: {nome}  ➜  {caminho_dst_parquet}")

        del df_temp # Free up memory
        gc.collect() # Run garbage collection

    except Exception as e:
        print(f"⚠️ Erro ao processar {caminho_src}: {e}")


# 3) Consolidate everything into a single Dask DataFrame (Bronze/current) by reading the Parquet files
if arquivos_csv:
    try:
        bronze_ddf = dask_cudf.read_parquet(os.path.join(BRONZE_RAW, "*/*/*.parquet"))

        # Convert 'ano' and 'mes' to Int64 as category dtype is not supported by parquet writer
        bronze_ddf["ano"] = bronze_ddf["ano"].astype("Int64")
        bronze_ddf["mes"] = bronze_ddf["mes"].astype("Int64")

        # Simple idempotence: remove exact duplicate rows
        bronze_ddf = bronze_ddf.drop_duplicates()

        print(f"\n🧱 Bronze consolidated: {len(bronze_ddf):,} linhas, {bronze_ddf.shape[1]} colunas")
        # For Dask, we use .head() to see the first few rows. This triggers computation.
        display(bronze_ddf.head())

        # 4) Salva consolidado (Parquet)
        out_parquet = os.path.join(BRONZE_CUR, "focos_bronze_consolidado.parquet")

        # To save as a single file, we repartition to 1 partition before saving
        bronze_ddf.repartition(npartitions=1).to_parquet(out_parquet)

        print(f"\n💾 Consolidado salvo em:\n - {out_parquet}")

    except Exception as e:
        print(f"⚠️ Erro ao consolidar ou salvar Bronze: {e}")
        # Handle the case where no parquet files were created
        if "Empty dataset" in str(e):
             print("No parquet files were created. Please check the download and processing steps.")

else:
    print("\nSkipping consolidation and saving Bronze as no CSV files were downloaded or processed.")

📦 Attempting to download all files from shared Google Drive folder: https://drive.google.com/drive/folders/1hRgLe3t-zGgWqFH7ZAtW0m6XQdgOW3bh?usp=sharing
Retrieving folder contents
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'drive.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Retrieving folder 1ovB3SBy-8dycREpOpS6R8IvZf1M9Pucf silver
Processing file 1C0kSP8H1texrVhDk3phK3JpbFzobhOvn focos_mensal_br_202401.csv
Processing file 1A5R0ddrYc9h1mdC13BcZgionHE2DOBqG focos_mensal_br_202402.csv
Processing file 1FfQ3dMqa4P8nJsj6aduzmuRK7KTGIic9 focos_mensal_br_202403.csv
Processing file 1xPIWC_TV-mF3Va_N4x2DpXtzqmMv-Uxt focos_mensal_br_202404.csv
Processing file 1mqey7AzyRkuBped-T_Qx5hXl2EfE4WA0 focos_mensal_br_202405.csv
Processing file 1NMvoGVZM2txO2O_31i28oBEfABs2eT9A focos_mensal_b

,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp,origem_arquivo,ano,mes
17798,4eecd6e2-bcdb-3f2f-824a-87677cd72966,-7.82228,-38.19895,2024-01-06 16:17:00,NOAA-20,SANTA CRUZ DA BAIXA VERDE,PERNAMBUCO,Brasil,2612471,26,33,5.0,0.0,0.58,Caatinga,4.4,focos_mensal_br_202401.csv,2024,1
17803,9fbb25f9-797f-3edc-b91f-ff56034560cd,-7.75481,-35.12300,2024-01-06 16:17:00,NOAA-20,TRACUNHAÉM,PERNAMBUCO,Brasil,2615508,26,33,5.0,0.0,1.00,Mata Atlântica,6.0,focos_mensal_br_202401.csv,2024,1
17804,53dfb92a-db47-3603-a5f2-d7e337331ef7,-7.73185,-35.37730,2024-01-06 16:17:00,NOAA-20,BUENOS AIRES,PERNAMBUCO,Brasil,2602704,26,33,5.0,0.0,1.00,Mata Atlântica,4.6,focos_mensal_br_202401.csv,2024,1
17807,4ff8c609-21b0-3881-9367-8890f5786476,-7.67674,-35.21803,2024-01-06 16:17:00,NOAA-20,ALIANÇA,PERNAMBUCO,Brasil,2600708,26,33,5.0,0.0,1.00,Mata Atlântica,3.1,focos_mensal_br_202401.csv,2024,1
17816,2d9c0aa8-1081-3d9e-8516-9b71a2c96d94,-10.44005,-38.43617,2024-01-06 16:17:00,NOAA-20,NOVO TRIUNFO,BAHIA,Brasil,2923050,29,33,0.0,0.0,0.90,Caatinga,38.8,focos_mensal_br_202401.csv,2024,1



💾 Consolidado salvo em:
 - /content/projeto_pcb/dados/bronze/current/focos_bronze_consolidado.parquet


In [ ]:
bronze_ddf.info()

<class 'dask_cudf._expr.collection.DataFrame'>
Columns: 19 entries, id to mes
dtypes: object(8), float64(5), int64(6)

In [ ]:
bronze_ddf.isna().sum().compute()

id                            0
lat                           0
lon                           0
data_hora_gmt                 0
satelite                      0
municipio                     0
estado                        0
pais                          0
municipio_id                  0
estado_id                     0
pais_id                       0
numero_dias_sem_chuva    125852
precipitacao             125852
risco_fogo               125852
bioma                         4
frp                      374354
origem_arquivo                0
ano                           0
mes                           0
dtype: int64

In [ ]:
df_temp = None
del df_temp
gc.collect()

84

# SILVER

In [ ]:
hoje = datetime.date.today()
BASE = "/content/projeto_pcb"
BRONZE = "/content/projeto_pcb/dados/bronze/current"
SILVER = "/content/projeto_pcb/dados/silver"
os.makedirs(SILVER, exist_ok=True)

In [ ]:

paths = sorted(glob.glob(os.path.join(BRONZE, "*.parquet")))
assert paths, "Nenhum arquivo Parquet encontrado em bronze."

bronze_ddf = dd.read_parquet(paths[0]) # Assuming the consolidated file is the first one

esperadas = {"data_hora_gmt","satelite","municipio","estado"}
# Check for missing columns (Dask DataFrames have columns attribute)
faltando = esperadas - set(bronze_ddf.columns)
assert not faltando, f"Faltam colunas no dataset: {faltando}"

bronze_ddf["data_hora_gmt"] = bronze_ddf["data_hora_gmt"].astype('datetime64[ns, UTC]')

for c in ["satelite","municipio","estado"]:
      bronze_ddf[c] = bronze_ddf[c].astype(str).str.strip()

bronze_ddf = bronze_ddf.dropna(subset=["data_hora_gmt"])

# Identify and replace -999 with NaN in numeric columns
numeric_cols = bronze_ddf.select_dtypes(include=np.number).columns
# Use Dask's assign with a dictionary comprehension for replacement
replace_dict = {col: bronze_ddf[col].replace(-999, np.nan) for col in numeric_cols}
bronze_ddf = bronze_ddf.assign(**replace_dict)


# Calculate and print proportion of NaNs per column using Dask
nan_proportion_per_column = bronze_ddf.isnull().sum().compute() / len(bronze_ddf) * 100
print("\nProporção de NaN por coluna (%):")
print(nan_proportion_per_column)

# Calculate and print proportion of rows with NaN using Dask
rows_with_nan = bronze_ddf.isnull().any(axis=1).sum().compute()
proportion_rows_with_nan = rows_with_nan / len(bronze_ddf) * 100
print(f"\nProporção de linhas com NaN no total do dataset (%): {proportion_rows_with_nan:.2f}%")

bronze_ddf = bronze_ddf.dropna()

bronze_ddf = bronze_ddf.drop(columns=['id', 'lat','lon','municipio_id','estado_id','pais_id','pais','origem_arquivo', 'ano', 'mes'])


# Extract date components using Dask's .dt accessor
bronze_ddf["ano"] = bronze_ddf["data_hora_gmt"].dt.year.astype("Int64")
bronze_ddf["mes"] = bronze_ddf["data_hora_gmt"].dt.month.astype("Int64")
bronze_ddf["dia"] = bronze_ddf["data_hora_gmt"].dt.day.astype("Int64")

# carga que prof pediu
bronze_ddf["dt_carga"] = datetime.date.today().isoformat()

# Salva silver as Parquet
SILVER_FILE =  f"focos_silver_{datetime.date.today():%Y%m%d}.parquet"
silver_parquet = os.path.join(SILVER,SILVER_FILE)
os.makedirs(SILVER, exist_ok=True)

# Save as a single parquet file
bronze_ddf.repartition(npartitions=1).to_parquet(silver_parquet)


print("Silver salvo:", silver_parquet, "| linhas:", len(bronze_ddf))

display(bronze_ddf.head())


Proporção de NaN por coluna (%):
id                       0.000000
lat                      0.000000
lon                      0.000000
data_hora_gmt            0.000000
satelite                 0.000000
municipio                0.000000
estado                   0.000000
pais                     0.000000
municipio_id             0.000000
estado_id                0.000000
pais_id                  0.000000
numero_dias_sem_chuva    1.916553
precipitacao             1.180835
risco_fogo               2.209782
bioma                    0.000038
frp                      3.512460
origem_arquivo           0.000000
ano                      0.000000
mes                      0.000000
dtype: float64

Proporção de linhas com NaN no total do dataset (%): 6.23%
Silver salvo: /content/projeto_pcb/dados/silver/focos_silver_20251013.parquet | linhas: 9993721


,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,...,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp,origem_arquivo,ano,mes,dia,dt_carga
3104,389b3bb6-3868-37e5-a876-529a459f8fc1,-9.65154,-38.02531,2024-01-03 15:35:00+00:00,NOAA-20,SANTA BRÍGIDA,BAHIA,Brasil,2927606,29,...,12.0,0.0,1.0,Caatinga,13.3,focos_mensal_br_202401.csv,2024,1,3,2025-10-13
3107,6df5c208-46a0-33a6-bda0-dda77222446e,-9.30832,-35.68497,2024-01-03 15:35:00+00:00,NOAA-20,SÃO LUÍS DO QUITUNDE,ALAGOAS,Brasil,2708501,27,...,12.0,0.0,1.0,Mata Atlântica,18.1,focos_mensal_br_202401.csv,2024,1,3,2025-10-13
3109,fae6ad07-8a8c-3435-94f7-a8b0c68614e8,-9.28939,-37.27657,2024-01-03 15:35:00+00:00,NOAA-20,POÇO DAS TRINCHEIRAS,ALAGOAS,Brasil,2707206,27,...,11.0,1.23,1.0,Caatinga,11.5,focos_mensal_br_202401.csv,2024,1,3,2025-10-13
26240,93af20d9-2c83-30f5-9421-32848322bd50,-18.95870,-57.17450,2024-01-07 20:46:51+00:00,GOES-16,CORUMBÁ,MATO GROSSO DO SUL,Brasil,5003207,50,...,4.0,0.0,0.07,Pantanal,72.4,focos_mensal_br_202401.csv,2024,1,7,2025-10-13
3114,374f9649-46cf-312d-9d3a-4afce1f3bfed,-9.08814,-36.59794,2024-01-03 15:35:00+00:00,NOAA-20,TEREZINHA,PERNAMBUCO,Brasil,2615102,26,...,11.0,0.0,1.0,Mata Atlântica,6.3,focos_mensal_br_202401.csv,2024,1,3,2025-10-13


In [ ]:
bronze_ddf.isna().sum()

Dask Series Structure:
npartitions=1
ano         int64
satelite      ...
Dask Name: sum, 56 expressions
Expr=(IsNa(frame=Assign(frame=Assign(frame=Assign(frame=Assign(frame=DropnaFrame(frame=Assign(frame=DropnaFrame(frame=Assign(frame=Assign(frame=Assign(frame=Assign(frame=ReadParquetFSSpec(de9e635))))), subset=['data_hora_gmt']))))))))).sum()

In [ ]:
bronze_ddf = None
del bronze_ddf
gc.collect()

30

# GOLD

In [ ]:
GOLD = "/content/projeto_pcb/dados/gold"
os.makedirs(GOLD, exist_ok=True)

In [ ]:
import dask.dataframe as dd
import os
import glob
import datetime
import pandas as pd # Keep pandas for final output display if needed
import dask_cudf # Import dask_cudf

GOLD = "/content/projeto_pcb/dados/gold"
os.makedirs(GOLD, exist_ok=True)

padrao_silver = "focos_silver_*.parquet"   # adjustment for parquet files

paths = sorted(glob.glob(os.path.join(SILVER, padrao_silver)))
assert paths, f"Nenhum arquivo Parquet Silver encontrado em: {SILVER}"

# Read the silver parquet files into a Dask DataFrame
# Dask can read multiple files matching a pattern directly
ddf_silver = dask_cudf.read_parquet(paths)

# Ensure 'data_hora_gmt' is datetime with UTC timezone
# ddf_silver["data_hora_gmt"] = ddf_silver["data_hora_gmt"].astype('datetime64[ns, UTC]') # Removed this line


# Normalizações úteis para agregação using Dask's .dt accessor
ddf_silver["ano"]  = ddf_silver["data_hora_gmt"].dt.year.astype("Int64")
ddf_silver["mes"]  = ddf_silver["data_hora_gmt"].dt.month.astype("Int64")
ddf_silver["dia"]  = ddf_silver["data_hora_gmt"].dt.day.astype("Int64")
ddf_silver["hora"] = ddf_silver["data_hora_gmt"].dt.hour.astype("Int64") # Convert to Int64

# Recreate 'data' column as a string for grouping
ddf_silver["data"] = ddf_silver["data_hora_gmt"].dt.strftime('%Y-%m-%d')


# Aggregations using Dask groupby
por_estado_mes = (
    ddf_silver.groupby(["estado","ano","mes"])
      .size()
      .reset_index()
      .rename(columns={0: "qtd_focos"}) # Rename the column after reset_index
      .compute() # Trigger computation and get pandas DataFrame
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)


por_municipio_dia = (
    ddf_silver.groupby(["estado","municipio","data"])
      .size()
      .reset_index()
      .rename(columns={0: "qtd_focos"}) # Rename the column after reset_index
      .compute() # Trigger computation and get pandas DataFrame
      .sort_values(["data","qtd_focos"], ascending=[True,False])
)


por_satelite_mes = (
    ddf_silver.groupby(["satelite","ano","mes"])
      .size()
      .reset_index()
      .rename(columns={0: "qtd_focos"}) # Rename the column after reset_index
      .compute() # Trigger computation and get pandas DataFrame
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)

por_satelite_total = (
    ddf_silver.groupby("satelite")
      .size()
      .reset_index()
      .rename(columns={0: "qtd_focos"}) # Rename the column after reset_index
      .compute() # Trigger computation and get pandas DataFrame
      .assign(perc=lambda x: (x["qtd_focos"] / x["qtd_focos"].sum())*100)
      .sort_values("qtd_focos", ascending=False)
      .round({"perc": 2})
)

# For spatial grid, compute first as it might be a large result
# Round lat/lon using Dask map_partitions if needed for large data
# ddf_silver["lat_cell"] = ddf_silver["lat"].map_partitions(lambda x: x.round(2))
# ddf_silver["lon_cell"] = ddf_silver["lon"].map_partitions(lambda x: x.round(2))
# Simple round for now, assuming it fits in memory after aggregation
ddf_silver["lat_cell"] = ddf_silver["lat"].round(2)
ddf_silver["lon_cell"] = ddf_silver["lon"].round(2)


grade_espacial = (
    ddf_silver.groupby(["lat_cell","lon_cell"])
      .size()
      .reset_index()
      .rename(columns={0: "qtd_focos"}) # Rename the column after reset_index
      .compute() # Trigger computation and get pandas DataFrame
      .sort_values("qtd_focos", ascending=False)
)

# ------------------------------
# Salva em GOLD
# ------------------------------
hoje = datetime.date.today().isoformat()

g1 = os.path.join(GOLD, f"gold_focos_por_estado_mes_{hoje}.parquet")
g2 = os.path.join(GOLD, f"gold_focos_por_municipio_dia_{hoje}.parquet")
g3 = os.path.join(GOLD, f"gold_focos_por_satelite_mes_{hoje}.parquet")
g3b= os.path.join(GOLD, f"gold_focos_por_satelite_total_{hoje}.parquet")
g4 = os.path.join(GOLD, f"gold_grade_espacial_{hoje}.parquet")

# Save pandas DataFrames to parquet
por_estado_mes.to_parquet(g1, index=False)
por_municipio_dia.to_parquet(g2, index=False)
por_satelite_mes.to_parquet(g3, index=False)
por_satelite_total.to_parquet(g3b, index=False)
grade_espacial.to_parquet(g4, index=False)


print("Gold salvo:")
print(" -", g1)
print(" -", g2)
print(" -", g3)
print(" -", g3b)
print(" -", g4)

try:
    ult_ano  = por_estado_mes["ano"].max()
    ult_mes  = por_estado_mes.query("ano == @ult_ano")["mes"].max()
    top_estados = (
        por_estado_mes
        .query("ano == @ult_ano and mes == @ult_mes")
        .nlargest(10, "qtd_focos")
    )
    print(f"\nTOP estados no mês mais recente ({ult_ano}-{ult_mes:02d}):")
    print(top_estados.to_string())
except Exception as e:
    print("\n[Aviso] Não foi possível imprimir TOP estados do mês mais recente:", e)

print("\n% por satelite (total):")
print(por_satelite_total.to_string())

Gold salvo:
 - /content/projeto_pcb/dados/gold/gold_focos_por_estado_mes_2025-10-13.parquet
 - /content/projeto_pcb/dados/gold/gold_focos_por_municipio_dia_2025-10-13.parquet
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_mes_2025-10-13.parquet
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_total_2025-10-13.parquet
 - /content/projeto_pcb/dados/gold/gold_grade_espacial_2025-10-13.parquet

TOP estados no mês mais recente (2025-10):
                 estado   ano  mes  qtd_focos
125            MARANHÃO  2025   10      57887
218           TOCANTINS  2025   10      55587
8                 GOIÁS  2025   10      46353
566               PIAUÍ  2025   10      44843
531               BAHIA  2025   10      37219
99         MINAS GERAIS  2025   10      33428
547         MATO GROSSO  2025   10      33184
175                PARÁ  2025   10      19456
489                ACRE  2025   10       9619
82   MATO GROSSO DO SUL  2025   10       9395

% por satelite (total):
    sateli

In [ ]:
# cont = df["Grey"].value_counts().sort_index()
# plt.bar(cont.index.astype(str), cont.values)
# plt.title("Distribuição da classe Grey")
# plt.xlabel("Grey (0=não cinza, 1=cinza)")
# # plt.ylabel("n")
# plt.show()


In [ ]:
# agg = df.groupby("Grey")[["R","G","B"]].mean().reset_index()
# for ch in ["R","G","B"]:
#     plt.bar(agg["Grey"].astype(str), agg[ch], label=ch)
#     plt.title(f"Média do canal {ch} por Grey")
#     plt.xlabel("Grey")
#     plt.ylabel(f"Média {ch} (escala {'0–255' if df['R'].max()>1 else '0–1'})")
#     plt.show()
